#Optimizer multiple evaluation

Perform multiple evaluation given some parameters.

Info:
1.   Make sure this colab is inside *scripts* in the main folder
2.   Make sure to have a folder called *results* and *logs/evaluation_logs* in the main folder 
3.   Run this on Colab on GPU or the time data is not comparable

**1. Setting Colab enviroments**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import subprocess
import sys
import os
from tqdm import tqdm
os.getcwd()

'/content'

In [3]:
cd /content/gdrive/My Drive/Colab Notebooks/Adversarial/AdvAttack/scripts

/content/gdrive/My Drive/Colab Notebooks/Adversarial/AdvAttack/scripts


In [4]:
print(os.getcwd())
print(os.listdir())

/content/gdrive/My Drive/Colab Notebooks/Adversarial/AdvAttack/scripts
['test.py', 'evaluation.py', 'Grid_evaluation.ipynb']


**2. Define params**

First we define some fixed parameters like the *optimizer*, the *dataset* etc..., then we create a list of dictionaries used inside the cycle.

**N.B** <br>
Cifar (since we call VGG) is much slower than MNIST even if it is easier to attack. I use as base number of examples 100, but when the attacks start to become too slow, like in case of target attack where you perform 9 attacks per example or when *n_gradients* start to become really high, I decrease the number of example to 50/20 (use only 20 for target attack!).

In [0]:
OPTIM = 'classic'
LOSS = '-1'
DATA = 'cifar10'
EPSILON = '0.01'
MAXIMISE = '1'
N_EXAMPLE = '20'
ALPHA = '0.2'
EPOCHS = '100'

MAX_BATCH_SIZE = 4000

params_list = [
               {'n_gradient':  500},
               {'n_gradient':  1000},
               {'n_gradient':  2000},
               {'n_gradient':  4000},
               {'n_gradient':  8000},
               ]

**3. Run the evaluation.py script**

We insert manually the fixed params in *exe_list* than we insert automatically the specific cycle parameters. Finally we call *evaluate.py*

In [0]:
for params in tqdm(params_list):
    exe_list = ['python', 'evaluation.py', 
                '--optimizer', OPTIM, 
                '--data', DATA, 
                '--epsilon', EPSILON, 
                '--L_type', LOSS, 
                '--alpha', ALPHA, 
                '--maximise', MAXIMISE, 
                '--n_example', N_EXAMPLE,
                '--data_batch_size', N_EXAMPLE,
                '--epochs', EPOCHS]

    if params['n_gradient'] > MAX_BATCH_SIZE:
        exe_list.append('--batch_size')
        exe_list.append(str(MAX_BATCH_SIZE))
    
    for k in params:
        exe_list.append('--{}'.format(k))
        exe_list.append(str(params[k]))
    
    print(exe_list)
    out = subprocess.run(exe_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)

  0%|          | 0/5 [00:00<?, ?it/s]

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '1', '--n_example', '20', '--data_batch_size', '20', '--epochs', '100', '--n_gradient', '500']


 20%|██        | 1/5 [20:29<1:21:57, 1229.27s/it]

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '1', '--n_example', '20', '--data_batch_size', '20', '--epochs', '100', '--n_gradient', '1000']


 40%|████      | 2/5 [59:00<1:17:41, 1553.82s/it]

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '1', '--n_example', '20', '--data_batch_size', '20', '--epochs', '100', '--n_gradient', '2000']


 60%|██████    | 3/5 [2:07:57<1:17:37, 2328.77s/it]

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '1', '--n_example', '20', '--data_batch_size', '20', '--epochs', '100', '--n_gradient', '4000']


**4. See results**

Here we open the logs and store the logs having specific values in their dictionaries, than we create a dataframe and save it in results.

**N.B** <br>
Different optimizer needs to take in consideration different parameters. Make sure to have as columns in the dataframe the columns *Succes Rate*, *Time per attack* and then your non-fixed parameters (e.g. *n_gradient*).


In [34]:
import json
import pandas as pd


path = '../logs/evaluation_logs'
r = []
for file_name in os.listdir(path):
    with open('{}/{}'.format(path, file_name), 'r') as file:
        logs = json.load(file)
        if logs['All_config_args']['epsilon'] ==  float(EPSILON) and logs['Dataset'] ==  DATA and logs['L norm'] ==  int(LOSS) and logs['Optimizer'] ==  OPTIM and logs['Target attack'] == int(MAXIMISE):
            print(logs)
            a, n, v = logs['All_config_args']['alpha'], logs['All_config_args']['n_gradient'], logs['All_config_args']['v']
            s, t = logs['Results']['Success'], logs['Results']['Mean time']
            r.append([s, t, n, a, v])


df = pd.DataFrame(r)
df.columns = ['Succes Rate', 'Time per attack', '# Gradient Estimation', 'Alpha', 'Gaussian Smoothing']


attack_type = 'target' if MAXIMISE == '1' else 'untarget'
res_path = '../results/{}_{}_{}_{}.csv'.format(OPTIM, DATA, attack_type, str(EPSILON)[2:])
print('Results saved at {}'.format(res_path))
df.to_csv(res_path)


df.sort_values(by=['Succes Rate', 'Time per attack'], axis=0, ascending=False)

{'Dataset': 'cifar10', 'Inception': False, 'Optimizer': 'classic', 'Loss': 'Zoo', 'L norm': -1, 'Target attack': 1, 'Results': {'Success': 0.9555555555555556, 'Mean time': 9.286920605765449, 'Mean epsilon': 0.015177709981799126}, 'All_config_args': {'seed': 42, 'no_cuda': False, 'data': 'cifar10', 'root': '../data/', 'transform': 'standard', 'data_batch_size': 10, 'num_workers': 0, 'n_example': 10, 'use_inception': False, 'model_path': 'none', 'loss': 'Zoo', 'maximise': 1, 'is_softmax': False, 'softmax_dim': 1, 'optimizer': 'classic', 'C': [0, 1], 'L_type': -1, 'epsilon': 0.02, 'verbose': 0, 'epochs': 100, 'max_time': 1000, 'tqdm_disabled': 1, 'v': 0.001, 'n_gradient': 4000, 'batch_size': -1, 'alpha': 0.2, 'gamma': 2, 'mu': 0.0025, 'max_t': 50, 'lr': 0.1, 'logs_path': '../logs/evaluation_logs'}}
{'Dataset': 'cifar10', 'Inception': False, 'Optimizer': 'classic', 'Loss': 'Zoo', 'L norm': -1, 'Target attack': 1, 'Results': {'Success': 0.4888888888888889, 'Mean time': 4.527152215109931, 'M

,Succes Rate,Time per attack,# Gradient Estimation,Alpha,Gaussian Smoothing
0,0.955556,9.286921,4000,0.2,0.001
4,0.944444,19.394981,8000,0.2,0.001
3,0.777778,9.864016,2000,0.2,0.001
2,0.666667,6.401701,1000,0.2,0.001
1,0.488889,4.527152,500,0.2,0.001
